# MMD-MP


#### Citation
@inproceedings{gao2020maximum,
  title={Maximum Mean Discrepancy is Aware of Adversarial Attacks},
  author={Gao, Ruize and Liu, Feng and Zhang, Jingfeng and Han, Bo and Liu, Tongliang and Niu, Gang and Sugiyama, Masashi},
  booktitle={ICML},
  year={2021}
}

In [ ]:
import argparse
import os
import numpy as np
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable
import torch.nn as nn
import torch
from scipy import signal

# Setup seeds
os.makedirs("images", exist_ok=True)
np.random.seed(819)
torch.manual_seed(819)
torch.cuda.manual_seed(819)
torch.backends.cudnn.deterministic = True
is_cuda = True

In [ ]:
n_epochs = 2000
batch_size = 100
lr = 0.0002
img_size = 64
channels = 3
n = 500
dtype = torch.float
device = torch.device("cuda:0")
cuda = True if torch.cuda.is_available() else False
N_per = 100 # permutation times
alpha = 0.05 # test threshold
N1 = n # number of samples in one set
K = 20 # number of trails
N = 100 # number of test sets
N_f = 100.0 # number of test sets (float)
is_yy_zero = True
is_xx_zero = False
sigmaG = 1000
coeff_xy = 2

In [ ]:
def get_item(x, is_cuda):
    """get the numpy value from a torch tensor."""
    if is_cuda:
        x = x.cpu().detach().numpy()
    else:
        x = x.detach().numpy()
    return x

In [ ]:
#@title PDist2
def Pdist2(x, y):
    """compute the paired distance between x and y."""
    x_norm = (x ** 2).sum(1).view(-1, 1)
    if y is not None:
        y_norm = (y ** 2).sum(1).view(1, -1)
    else:
        y = x
        y_norm = x_norm.view(1, -1)
    Pdist = x_norm + y_norm - 2.0 * torch.mm(x, torch.transpose(y, 0, 1))
    Pdist[Pdist<0]=0
    return Pdist

In [ ]:
#@title h1_mean_var_gram
def h1_mean_var_gram(Kx, Ky, Kxy, is_var_computed, use_1sample_U=True, is_unbiased = True, coeff_xy=2, is_yy_zero=False, is_xx_zero=False):
	"""compute value of MMD and std of MMD using kernel matrix."""
	if not is_yy_zero:
		coeff_yy = 1
	else:
		coeff_yy = 0
	if not is_xx_zero:
		coeff_xx = 1
	else:
		coeff_xx = 0
	Kxxy = torch.cat((Kx,Kxy),1)
	Kyxy = torch.cat((Kxy.transpose(0,1),Ky),1)
	Kxyxy = torch.cat((Kxxy,Kyxy),0)
	nx = Kx.shape[0]
	ny = Ky.shape[0]

	if is_unbiased:
		xx = torch.div((torch.sum(Kx) - torch.sum(torch.diag(Kx))), (nx * (nx - 1)))
		yy = torch.div((torch.sum(Ky) - torch.sum(torch.diag(Ky))), (ny * (ny - 1)))
		# one-sample U-statistic.
		if use_1sample_U:
			xy = torch.div((torch.sum(Kxy) - torch.sum(torch.diag(Kxy))), (nx * (ny - 1)))
		else:
			xy = torch.div(torch.sum(Kxy), (nx * ny))
		mmd2 = xx * coeff_xx - coeff_xy * xy + yy * coeff_yy
	else:
		xx = torch.div((torch.sum(Kx)), (nx * nx))
		yy = torch.div((torch.sum(Ky)), (ny * ny))
		# one-sample U-statistic.
		if use_1sample_U:
			xy = torch.div((torch.sum(Kxy)), (nx * ny))
		else:
			xy = torch.div(torch.sum(Kxy), (nx * ny))
		mmd2 = xx * coeff_xx - coeff_xy * xy + yy * coeff_yy
	if not is_var_computed:
		return mmd2, None, Kxyxy
	hh = Kx*coeff_xx+Ky * coeff_yy-(Kxy+Kxy.transpose(0,1))*coeff_xy/2
	V1 = torch.dot(hh.sum(1)/ny,hh.sum(1)/ny) / ny
	V2 = (hh).sum() / (nx) / nx
	varEst = 4*(V1 - V2**2)
	if  varEst == 0.0:
		print('error_var!!'+str(V1))
	return mmd2, varEst, Kxyxy

In [ ]:
#@title MatConvert
def MatConvert(x, device, dtype):
    """convert the numpy to a torch tensor."""
    x = torch.from_numpy(x).to(device, dtype)
    return x

In [ ]:
#@title MMD-MP Function
def MMDu(Fea, len_s, Fea_org, sigma, sigma0=0.1, epsilon = 10**(-10), is_smooth=True, is_var_computed=True, use_1sample_U=True, is_unbiased=True, coeff_xy=2, is_yy_zero=False, is_xx_zero=False):
	X = Fea[0:len_s, :] # fetch the sample 1 (features of deep networks)
	Y = Fea[len_s:, :] # fetch the sample 2 (features of deep networks)
	X_org = Fea_org[0:len_s, :] # fetch the original sample 1
	Y_org = Fea_org[len_s:, :] # fetch the original sample 2
	L = 1 # generalized Gaussian (if L>1)

	nx = X.shape[0]
	ny = Y.shape[0]
	Dxx = Pdist2(X, X)
	Dyy = Pdist2(Y, Y)
	Dxy = Pdist2(X, Y)
	Dxx_org = Pdist2(X_org, X_org)
	Dyy_org = Pdist2(Y_org, Y_org)
	Dxy_org = Pdist2(X_org, Y_org)
	K_Ix = torch.eye(nx).cuda()
	K_Iy = torch.eye(ny).cuda()
	if is_smooth:
		Kx = (1-epsilon) * torch.exp(-(Dxx / sigma0)**L -Dxx_org / sigma) + epsilon * torch.exp(-Dxx_org / sigma)
		Ky = (1-epsilon) * torch.exp(-(Dyy / sigma0)**L -Dyy_org / sigma) + epsilon * torch.exp(-Dyy_org / sigma)
		Kxy = (1-epsilon) * torch.exp(-(Dxy / sigma0)**L -Dxy_org / sigma) + epsilon * torch.exp(-Dxy_org / sigma)
	else:
		Kx = torch.exp(-Dxx / sigma0)
		Ky = torch.exp(-Dyy / sigma0)
		Kxy = torch.exp(-Dxy / sigma0)

	return h1_mean_var_gram(Kx, Ky, Kxy, is_var_computed, use_1sample_U,is_unbiased, coeff_xy, is_yy_zero, is_xx_zero)

In [ ]:
def TST_MMD_u(Fea, N_per, N1, Fea_org, sigma, sigma0, ep, alpha, device, dtype, is_smooth=True):
    """run two-sample test (TST) using deep kernel kernel."""
    mmd_vector = np.zeros(N_per)
    TEMP = MMDu(Fea, N1, Fea_org, sigma, sigma0, ep, is_smooth)
    mmd_value = get_item(TEMP[0], is_cuda)
    Kxyxy = TEMP[2]
    count = 0
    nxy = Fea.shape[0]
    nx = N1
    for r in range(N_per):
        # print r
        ind = np.random.choice(nxy, nxy, replace=False)
        # divide into new X, Y
        indx = ind[:nx]
        # print(indx)
        indy = ind[nx:]
        Kx = Kxyxy[np.ix_(indx, indx)]
        # print(Kx)
        Ky = Kxyxy[np.ix_(indy, indy)]
        Kxy = Kxyxy[np.ix_(indx, indy)]

        TEMP = h1_mean_var_gram(Kx, Ky, Kxy, is_var_computed=False)
        mmd_vector[r] = TEMP[0]
        if mmd_vector[r] > mmd_value:
            count = count + 1
        if count > np.ceil(N_per * alpha):
            h = 0
            threshold = "NaN"
            break
        else:
            h = 1
    if h == 1:
        S_mmd_vector = np.sort(mmd_vector)
        #        print(np.int(np.ceil(N_per*alpha)))
        threshold = S_mmd_vector[np.int64(np.ceil(N_per * (1 - alpha)))]
    return h, threshold, mmd_value.item()

In [ ]:
def TST_MMD_u_WB(Fea, N_per, N_te, Fea_org, sigma, sigma0, epsilon, alpha, device, dtype):

    X = Fea[0:N_te, :] # fetch the sample 1 (features of deep networks)
    Y = Fea[N_te:, :] # fetch the sample 2 (features of deep networks)
    X_org = Fea_org[0:N_te, :] # fetch the original sample 1
    Y_org = Fea_org[N_te:, :] # fetch the original sample 2


    Dxx = Pdist2(X, X)
    Dyy = Pdist2(Y, Y)
    Dxy = Pdist2(X, Y)
    Dxx_org = Pdist2(X_org, X_org)
    Dyy_org = Pdist2(Y_org, Y_org)
    Dxy_org = Pdist2(X_org, Y_org)

    Kx = (1-epsilon) * torch.exp(-(Dxx / sigma0) -Dxx_org / sigma) + epsilon * torch.exp(-Dxx_org / sigma)
    Ky = (1-epsilon) * torch.exp(-(Dyy / sigma0) -Dyy_org / sigma) + epsilon * torch.exp(-Dyy_org / sigma)
    Kxy = (1-epsilon) * torch.exp(-(Dxy / sigma0) -Dxy_org / sigma) + epsilon * torch.exp(-Dxy_org / sigma)

    return bootsrap_null(N_te, N_per, Kx, Ky, Kxy, alpha, device, dtype)

In [ ]:
#@title wild bootstrap process
def Wild_bootsrap_process(data_length, Num_trials):
    #ln = 20.0
    #ln = 0.2 原参数0.2
    ln = 0.2
    ar = np.exp(-1/ln)
    variance = 1-np.exp(-2/ln)

    w = np.sqrt(variance) * np.random.randn(data_length, Num_trials)
    a = np.array([1,-1 * ar])
    process = signal.lfilter(np.array([1]), a, w)
    return process


In [ ]:
#@title bootstrap_null
def bootsrap_null(data_length, Num_trials, Kx, Ky, Kxy, alpha, device, dtype):

    process = MatConvert(Wild_bootsrap_process(data_length, Num_trials), device, dtype)
    testStatMat = Kx + Ky - 2*Kxy
    testStat = testStatMat.mean()
    testStat_tem = torch.zeros(Num_trials).to(device, dtype)
    count = 0
    for kk in range(Num_trials):
        mn = process[:,kk].mean()
        matWB = (process[:,kk] - mn).unsqueeze(1).matmul((process[:,kk] - mn).unsqueeze(0))
        testStat_tem[kk] = (testStatMat * matWB).mean()
        if testStat_tem[kk] > testStat:
            count = count + 1
        if count > np.ceil(Num_trials * alpha):
            h = 0
            threshold = "NaN"
            break
        else:
            h = 1
    if h == 1:
        S_mmd_vector = np.sort(testStat_tem.cpu().detach().numpy())
        #        print(np.int(np.ceil(N_per*alpha)))
        threshold = S_mmd_vector[np.int64(np.ceil(Num_trials * (1 - alpha)))]
    return h, threshold, testStat.item()

In [ ]:
def TST_MMD_u_WB_old(Fea, N_per, N1, Fea_org, sigma, sigma0, ep, alpha, device, dtype, is_smooth=True):
    """run two-sample test (TST) using deep kernel kernel."""
    mmd_vector = np.zeros(N_per)
    TEMP = MMDu(Fea, N1, Fea_org, sigma, sigma0, ep, is_smooth)
    mmd_value = get_item(TEMP[0], is_cuda)
    Kxyxy = TEMP[2]
    count = 0
    nxy = Fea.shape[0]
    nx = N1
    for r in range(N_per):
        # print r
        ind = np.random.choice(nxy, nxy, replace=False)
        # divide into new X, Y
        indx = ind[:nx]
        # print(indx)
        indy = ind[nx:]
        Kx = Kxyxy[np.ix_(indx, indx)]
        # print(Kx)
        Ky = Kxyxy[np.ix_(indy, indy)]
        Kxy = Kxyxy[np.ix_(indx, indy)]

        TEMP = bootsrap_null(N_te, N_per, Kx, Ky, Kxy, alpha, device, dtype)
        mmd_vector[r] = TEMP[0]
        if mmd_vector[r] > mmd_value:
            count = count + 1
        if count > np.ceil(N_per * alpha):
            h = 0
            threshold = "NaN"
            break
        else:
            h = 1
    if h == 1:
        S_mmd_vector = np.sort(mmd_vector)
        #        print(np.int(np.ceil(N_per*alpha)))
        threshold = S_mmd_vector[np.int64(np.ceil(N_per * (1 - alpha)))]
    return h, threshold, mmd_value.item()

In [ ]:
#@title Model
class Featurizer(nn.Module):
    def __init__(self):
        super(Featurizer, self).__init__()

        def discriminator_block(in_filters, out_filters, bn=True):
            block = [nn.Conv2d(in_filters, out_filters, 3, 2, 1), nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0)] #0.25
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block

        self.model = nn.Sequential(
            *discriminator_block(channels, 16, bn=False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128),
        )

        # The height and width of downsampled image
        ds_size = img_size // 2 ** 4
        self.adv_layer = nn.Sequential(
            nn.Linear(512, 300))

    def forward(self, img):
        out = self.model(img)
        out = out.view(out.shape[0], -1)
        feature = self.adv_layer(out)

        return feature


In [ ]:
#@title load data
transform_test = transforms.Compose([transforms.ToTensor(),])
testset = datasets.CIFAR10(root='./data/cifar10', train=False, download=True, transform=transform_test)
test_loader = torch.utils.data.DataLoader(testset, batch_size=10000, shuffle=False, num_workers=0)

print(test_loader)
# Obtain CIFAR10 images
for i, (imgs, Labels) in enumerate(test_loader):
    Cifar_data_all = imgs
    label_all = Labels

ind_Cifar = np.random.choice(len(Cifar_data_all), len(Cifar_data_all), replace=False)
Cifar_data_all = Cifar_data_all[ind_Cifar]

data_all=Cifar_data_all[2000:]

Ind_all = np.arange(len(data_all))

'''
^
Downloads test data, puts it into an dataloader with batch size equivalent to dataset size (10,000).
Splits it into labels and imgs.
Randomized the order. Ind_Cifar is a list of randomized indexes, which gets inputed back in the data.
Removes the first 2000 images from the set.
Generates index for that final data
'''


#download adversarial data
data_trans = np.load('./drive/MyDrive/Uni/Topics/True_Adv_Cifar10_PGD20_eps8.npy')
ind_Cifar = np.random.choice(len(data_trans), len(data_trans), replace=False)
data_trans = data_trans[ind_Cifar]
data_trans = data_trans[0:1000]

data_trans=torch.from_numpy(data_trans).float()

Ind_data = np.random.choice(len(data_trans), len(data_trans), replace=False)
data_trans = data_trans[Ind_data]

Ind_v4_all = np.arange(len(data_trans))

'''
^
Loads adversarial data into variable data_trans.
Shuffles the data by making an index list from random.choice.
Keeps only the first 1000 images.
Converts numpy to tensor
Shuffles those 1000 images.
Then gets the index of them.
'''

100%|██████████| 170498071/170498071 [00:10<00:00, 15584583.10it/s]


Extracting ./data/cifar10/cifar-10-python.tar.gz to ./data/cifar10


'\n^\nLoads adversarial data into variable data_trans.\nShuffles the data by making an index list from random.choice.\nKeeps only the first 1000 images.\nConverts numpy to tensor\nShuffles those 1000 images.\nThen gets the index of them.\n'

In [ ]:
def test_accuracy():
  H_u = np.zeros(N)
  # Fetch test data
  data_all_te = data_all[Ind_te]
  N_te = len(data_trans) - N1
  Ind_N_te = np.random.choice(len(Ind_te), N_te, replace=False)#9900
  s1 = data_all_te[Ind_N_te]
  s2 = data_trans[Ind_te_v4]
  S = torch.cat([s1.cpu(), s2.cpu()], 0).cuda()
  Sv = S.view(2 * N_te, -1)

  #print(sigma0_u)
  # MMD-D
  h_u, threshold_u, mmd_value_u = TST_MMD_u(featurizer(S), N_per, N_te, Sv, sigma, sigma0_u, ep, alpha, device, dtype)
  return h_u


In [ ]:
ep_OPT = np.zeros([K])
s_OPT = np.zeros([K])
s0_OPT = np.zeros([K])
'''
kernel k_w
w or omega or parameters = {epsilon, sigma, sigma 0}
'''

TruePositive = np.zeros([2, K])
TrueNegative = np.zeros([2, K])
for kk in range(K):
    torch.manual_seed(kk * 19 + N1)
    torch.cuda.manual_seed(kk * 19 + N1)
    np.random.seed(seed=1102 * (kk + 10) + N1)

    # Initialize deep networks for MMD-D (called featurizer), C2ST-S and C2ST-L (called discriminator)
    featurizer = Featurizer()

    # Initialize parameters
    epsilonOPT = torch.log(MatConvert(np.random.rand(1) * 10 ** (-10), device, dtype))
    epsilonOPT.requires_grad = True
    sigmaOPT = MatConvert(np.ones(1) * np.sqrt(2 * 32 * 32), device, dtype)
    sigmaOPT.requires_grad = True
    sigma0OPT = MatConvert(np.ones(1) * np.sqrt(0.005), device, dtype)
    sigma0OPT.requires_grad = True
    #print(epsilonOPT.item())
    if cuda:
        featurizer.cuda()
    print(len(data_all))

    # Collect natural images
    Ind_tr = np.random.choice(len(data_all), N1, replace=False)
    Ind_te = np.delete(Ind_all, Ind_tr)
    train_data = []
    for i in Ind_tr:
       train_data.append([data_all[i], label_all[i]])


    dataloader = torch.utils.data.DataLoader(
        train_data,
        batch_size=batch_size,
        shuffle=True,
    )

    '''
    Intializes featurizer
    Main parameters to be adjusted
    Loads in natural data for training and testing
    '''

    # Collect adv images
    np.random.seed(seed=819 * (kk + 9) + N1)
    Ind_tr_v4 = np.random.choice(len(data_trans), N1, replace=False)
    Ind_te_v4 = np.delete(Ind_v4_all, Ind_tr_v4)
    New_CIFAR_tr = data_trans[Ind_tr_v4]
    New_CIFAR_te = data_trans[Ind_te_v4]

    # Initialize optimizers
    optimizer_F = torch.optim.Adam(list(featurizer.parameters()) + [epsilonOPT] + [sigmaOPT] + [sigma0OPT], lr=lr)


    Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

    # ----------------------------------------------------------------------------------------------------
    #  Training deep networks for MMD-D (called featurizer), C2ST-S and C2ST-L (called discriminator)
    # ----------------------------------------------------------------------------------------------------
    np.random.seed(seed=1102)
    torch.manual_seed(1102)
    torch.cuda.manual_seed(1102)
    batch_loss = []
    for epoch in range(n_epochs):
        for i, (imgs, _) in enumerate(dataloader):
            if True:
                ind = np.random.choice(N1, imgs.shape[0], replace=False)
                Fake_imgs = New_CIFAR_tr[ind]
                # Adversarial ground truths
                valid = Variable(Tensor(imgs.shape[0], 1).fill_(1.0), requires_grad=False)
                fake = Variable(Tensor(imgs.shape[0], 1).fill_(0.0), requires_grad=False)

                # Configure input
                real_imgs = Variable(imgs.type(Tensor))
                Fake_imgs = Variable(Fake_imgs.type(Tensor))
                X = torch.cat([real_imgs, Fake_imgs], 0)

                '''
                X contains both real and fake images. Fake images come from adverserial images.
                '''

                # ------------------------------
                #  Train deep network for MMD-D
                # ------------------------------
                # Initialize optimizer
                optimizer_F.zero_grad()
                # Compute output of deep network

                modelu_output = featurizer(X)
                #print(modelu_output[0][0])
                # Compute epsilon, sigma and sigma_0
                ep = torch.exp(epsilonOPT) / (1 + torch.exp(epsilonOPT))
                sigma = sigmaOPT ** 2
                sigma0_u = sigma0OPT ** 2
                # Compute Compute J (STAT_u)
                TEMP = MMDu(modelu_output, imgs.shape[0], X.view(X.shape[0],-1), sigma, sigma0_u, ep, coeff_xy=coeff_xy, is_yy_zero=is_yy_zero, is_xx_zero=is_xx_zero)
                mmd_value_temp = -1 * TEMP[0]
                #mmd_value_temp = TEMP[0]
                mmd_std_temp = torch.sqrt(TEMP[1] + 10 ** (-8))
                STAT_u = torch.div(mmd_value_temp, mmd_std_temp)
                #STAT_u_new = 1/STAT_u
                # Compute gradient
                #STAT_u_new.backward()
                STAT_u.backward()
                # Update weights using gradient descent
                optimizer_F.step()
                batch_loss.append(STAT_u.item())
            else:
                break
        if (epoch % 100 == 0):
                print(f"epoch:{epoch+1}, mmd_value_temp:{mmd_value_temp.item()}, STAT_u:{np.average(batch_loss)}")
                batch_loss = []


    # Run two-sample test on the training set
    # Fetch training data
    s1 = data_all[Ind_tr]
    s2 = data_trans[Ind_tr_v4]
    S = torch.cat([s1.cpu(), s2.cpu()], 0).cuda()
    Sv = S.view(2 * N1, -1)
    # Run two-sample test (MMD-D) on the training set
    h_u, threshold_u, mmd_value_u = TST_MMD_u(featurizer(S), N_per, N1, Sv, sigma, sigma0_u, ep, alpha, device, dtype)

    # Record best epsilon, sigma and sigma_0
    ep_OPT[kk] = ep.item()
    s_OPT[kk] = sigma.item()
    s0_OPT[kk] = sigma0_u.item()

    # Compute test power of MMD-D and baselines
    TP_u = np.zeros(N)
    TN_u = np.zeros(N)
    #T_u = np.zeros(N)
    #M_u = np.zeros(N)
    TP_wb = np.zeros(N)
    TN_wb = np.zeros(N)
    #T_wb = np.zeros(N)
    #M_wb = np.zeros(N)

    np.random.seed(1102)
    count_u = 0
    count_wb = 0
    for k in range(N):
        # Fetch test data
        np.random.seed(seed=1102 * (k + 1) + N1)
        data_all_te = data_all[Ind_te]
        N_te = len(data_trans) - N1
        Ind_N_te = np.random.choice(len(Ind_te), N_te, replace=False)#9900
        s1 = data_all_te[Ind_N_te]
        s2 = data_trans[Ind_te_v4]
        Stp = torch.cat([s1.cpu(), s2.cpu()], 0).cuda()
        Svtp = Stp.view(2 * N_te, -1)
        Ind_N_te = np.random.choice(len(Ind_te), N_te, replace=False)#9900
        s3 = data_all_te[Ind_N_te]
        Stn = torch.cat([s1.cpu(), s3.cpu()], 0).cuda()
        Svtn = Stn.view(2 * N_te, -1)

        #print(sigma0_u)
        # MMD-D
        tp_u, threshold_u, mmd_value_u = TST_MMD_u(featurizer(Stp), N_per, N_te, Svtp, sigma, sigma0_u, ep, alpha, device, dtype)
        tp_wb, threshold_wb, mmd_value_wb = TST_MMD_u_WB(featurizer(Stp), N_per, N_te, Svtp, sigma, sigma0_u, ep, alpha, device, dtype)

        tn_u = 1 - (TST_MMD_u(featurizer(Stn), N_per, N_te, Svtn, sigma, sigma0_u, ep, alpha, device, dtype)[0])
        tn_wb = 1 - (TST_MMD_u_WB(featurizer(Stn), N_per, N_te, Svtn, sigma, sigma0_u, ep, alpha, device, dtype)[0])

        # Gather results
        count_u = count_u + tp_u
        print("MMD-MP:", count_u)
        TP_u[k] = tp_u
        TN_u[k] = tn_u
        #T_u[k] = threshold_u
        #M_u[k] = mmd_value_u

        count_wb = count_wb + tp_wb
        print("MMD-MP_WB:", count_wb)
        TP_wb[k] = tp_wb
        TN_wb[k] = tn_wb
        #T_wb[k] = threshold_wb
        #M_wb[k] = mmd_value_wb


    # Print baselines
    print("Reject rate_u: ", TP_u.sum() / N_f)
    TruePositive[0, kk] = TP_u.sum() / N_f
    print("Test Power of Baselines (K times): ")
    print(TruePositive)
    print("Average Test Power of Baselines (K times): ")
    print("MMD-MP: ", (TruePositive.sum(1) / (kk + 1))[0])

    print("Reject rate_u: ", TP_wb.sum() / N_f)
    TruePositive[1, kk] = TP_wb.sum() / N_f
    print("Test Power of Baselines (K times): ")
    print(TruePositive)
    print("Average Test Power of Baselines (K times): ")
    print("MMD-MP_WB: ", (TruePositive.sum(1) / (kk + 1))[1])

    TrueNegative[0, kk] = TN_u.sum() / N_f
    print("True Negative of Baselines (K times): ")
    print(TrueNegative)
    print("Average True Negative of Baselines (K times): ")
    print("MMD-MP: ", (TrueNegative.sum(1) / (kk + 1))[0])

    TrueNegative[1, kk] = TN_wb.sum() / N_f
    print("True Negative of Baselines (K times): ")
    print(TrueNegative)
    print("Average True Negative of Baselines (K times): ")
    print("MMD-MP_WB: ", (TrueNegative.sum(1) / (kk + 1))[1])

#np.save('./Results_CIFAR10_' + str(N1) + 'MMD-MP', Results)

8000


<ipython-input-16-7915c0c42d59>:77: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:78.)
  valid = Variable(Tensor(imgs.shape[0], 1).fill_(1.0), requires_grad=False)
/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Streaming output truncated to the last 5000 lines.
MMD-MP: 0
MMD-MP_WB: 0
MMD-MP: 0
MMD-MP_WB: 0
MMD-MP: 0
MMD-MP_WB: 0
MMD-MP: 0
MMD-MP_WB: 0
MMD-MP: 0
MMD-MP_WB: 0
MMD-MP: 0
MMD-MP_WB: 0
MMD-MP: 0
MMD-MP_WB: 0
MMD-MP: 0
MMD-MP_WB: 0
MMD-MP: 0
MMD-MP_WB: 0
MMD-MP: 0
MMD-MP_WB: 0
MMD-MP: 0
MMD-MP_WB: 0
MMD-MP: 0
MMD-MP_WB: 0
MMD-MP: 0
MMD-MP_WB: 0
MMD-MP: 0
MMD-MP_WB: 0
MMD-MP: 0
MMD-MP_WB: 0
MMD-MP: 0
MMD-MP_WB: 0
MMD-MP: 0
MMD-MP_WB: 0
MMD-MP: 0
MMD-MP_WB: 0
MMD-MP: 0
MMD-MP_WB: 0
MMD-MP: 0
MMD-MP_WB: 0
MMD-MP: 0
MMD-MP_WB: 0
MMD-MP: 0
MMD-MP_WB: 0
MMD-MP: 0
MMD-MP_WB: 0
MMD-MP: 0
MMD-MP_WB: 0
MMD-MP: 0
MMD-MP_WB: 0
MMD-MP: 0
MMD-MP_WB: 0
MMD-MP: 0
MMD-MP_WB: 0
MMD-MP: 0
MMD-MP_WB: 0
MMD-MP: 0
MMD-MP_WB: 0
MMD-MP: 0
MMD-MP_WB: 0
MMD-MP: 0
MMD-MP_WB: 0
MMD-MP: 0
MMD-MP_WB: 0
MMD-MP: 0
MMD-MP_WB: 0
MMD-MP: 0
MMD-MP_WB: 0
MMD-MP: 0
MMD-MP_WB: 0
MMD-MP: 0
MMD-MP_WB: 0
MMD-MP: 0
MMD-MP_WB: 0
MMD-MP: 0
MMD-MP_WB: 0
MMD-MP: 0
MMD-MP_WB: 0
MMD-MP: 0
MMD-MP_WB: 0
MMD-MP: 0
MMD-MP_WB: 0
MMD-MP